In [34]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Falconsai/text_summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("Falconsai/text_summarization")

In [39]:
import pandas as pd
import datasets

file_name = '../data/processed/summarized_insights.csv'

dataset = datasets.load_dataset("csv", data_files=file_name)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [36]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["Combined Comments"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["Summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

/Users/jonathanhu/anaconda3/envs/tf_metal/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [37]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Split data into training and validation sets
train_inputs, val_inputs, train_targets, val_targets = train_test_split(tokenized_data['input_ids'], tokenized_data['labels'], test_size=0.1)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_targets))
val_dataset = tf.data.Dataset.from_tensor_slices((val_inputs, val_targets))

# Batch the datasets
batch_size = 4
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)


In [38]:
# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer)

# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=3)


TypeError: compile() got an unexpected keyword argument 'optimizer'